<a href="https://colab.research.google.com/github/Eric-Oliveira-ds/computer-vision/blob/main/Train_a_Custom_Keypoint_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>